In [124]:
import pandas as pd


def LOG_parser(log_file):
    with open(log_file) as f:
        txt = f.read()
    content = txt.split("Gaussian NBO Version 3.1")[-1].split("\n")
    npa_res = []
    nbo_res = []
    npa_summary = []
    for k in range(len(content)):
        line = content[k]
        if "NATURAL POPULATIONS" in line:
            for i in range(k+4, len(content)):
                if "WARNING" not in content[i]:
                    # print(content[i])
                    if content[i]:
                        info = content[i].split()
                        atom, occ, energy = info[1], float(info[-2]), float(info[-1])
                        npa_res.append((atom, occ, energy))
                else:
                    break
        elif "Summary of Natural Population Analysis:" in line:
            for i in range(k+6, len(content)):
                s = content[i]
                if "=" not in s:
                    atom = s.split()[0]
                    charge = float(s.split()[2])
                    npa_summary.append((atom, charge))
                else:
                    break
        elif "Natural Bond Orbitals (Summary):" in line:
            for i in range(k+6, len(content)):
                s = content[i]
                if "-------------------------------" not in s:
                    if s[6] == ".":
                        atom = s[18: 20].replace(" ", "")
                        occ = float(s[41: 48])
                        energy = float(s[48: 62])
                        nbo_res.append((atom, occ, energy))
                        # print(s)
                        # print(f"atom:{atom}, occupancy:{occ}, energy: {energy}")
                else:
                    break
    res = {"NPA": npa_res, "NPA_summary": npa_summary, "NBO": nbo_res}
    return res

In [125]:
data = {}
for t in "ABCDE":
    data[t] = {}
    d = LOG_parser(f"{t}.LOG")
    data[t]['NPA'] = sorted([item for item in d['NPA'] if item[0] == "S"],
                             key=lambda x: x[-1], reverse=True)
    data[t]['NPA_charge'] = [item[-1] for item in d['NPA_summary'] if item[0] == "S"][0]
    data[t]['NBO'] = sorted([item for item in d['NBO'] if item[0] == "S"],
                             key=lambda x: x[-1], reverse=True)

In [135]:
df = {}
for t in "ABCDE":
    NPA = data[t]["NPA"]
    charge = data[t]["NPA_charge"]
    NBO = data[t]["NBO"]
    df[t] = [charge] + [x[1] for x in NBO[:2]] + [x[2] for x in NBO[:2]]

In [136]:
df = pd.DataFrame(df)
df.to_csv("NBO.csv")
df

,A,B,C,D,E
0,-0.03814,-0.09174,-0.10015,-0.03782,-0.03357
1,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.00000,0.00000,0.00000,0.00000,0.00000
3,185.55608,185.84197,185.50260,185.55347,185.49834
4,18.22537,18.03942,18.35424,18.28720,18.23176
